In [1]:
import pandas as pd
from src.llm import agent
pd.set_option('display.max_colwidth', None)

df = pd.read_excel('data/zkusenosti.xlsx')
df2 = pd.read_excel('data/Produkty.xlsx')

cols = ['zkID', 'prodID', 'tagy', 'nadpis', 'zkusenost', 'lng']
lngs = ['pl', 'cz', 'hu', 'de', 'ro', 'es', 'ru', 'sk', 'en', 'it', 'us']

c:\Users\Dominik\Documents\zkusenost-updater\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Dominik\Documents\zkusenost-updater\src\llm.py:15: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


In [ ]:
filtered = [
    "bracelets_new",
    "eurasiapro",
    "extra",
    "astrafit",
    "hair color",
    "hodinky",
    "karta",
    "masážní",
    "nádobí",
    "reklama",
    "propagace",
    "samolepky",
    "sport-r",
    "termoterapie",
    "tiskoviny"
]
product_blacklist = [
    "DUO WHITE",
    "WHITE SET",
    "DUO WHITE",
    "WINDOW-CLEANER SET",
    "SNOW-BRUSH",
    "ICE-SCRAPER",
    "WHITE MAX",
    "WOOD MAX",
    "DUST MAX",
    "WINDOW-CLEANER"
    "WIRE TRIPLE STAR",
    "WIRE DOUBLE STAR",
    "SOAP mini",
    "SOAP WHITE",
    "SOAP BED",
]

import re
def extract_base_name(nazev):
    match = re.match(r'^(.*?)(\d+)$', nazev.strip())  # Extracts name and trailing number
    if match:
        return match.group(1).strip(), int(match.group(2))  # (Base Name, Number)
    return nazev.strip(), None  # Return name as-is if no number

df2_filtered = df2[
    (~df2['druh'].isin(filtered).fillna(False)) &  
    df2['show_it_CZ'].eq(1) & 
    ~df2['nazev'].str.endswith('SAMPLE', na=False) &  
    ~df2['nazev'].str.contains(r'\b\d+\s*[x×]\s*\d+(?:\s*cm)?\b', na=False) &
    ~df2['nazev'].str.match(r'^\w+$', na=False) & 
    ~df2['nazev'].str.contains("cm", case=True, na=False) &
    (~df2['nazev'].str.contains(",", case=False, na=False) | df2['nazev'].str.contains("CBD", case=False, na=False)) &
    ~df2['nazev'].str.contains("FLOOR", case=False, na=False) &
    ~df2['nazev'].str.contains("GLOVE", case=False, na=False) &
    ~df2['nazev'].isin(product_blacklist)
][['ID', 'nazev']]
df2_filtered = df2_filtered[~df2_filtered.duplicated(subset='nazev', keep='first')]
df2_filtered[['base_nazev', 'number']] = df2_filtered['nazev'].apply(lambda x: pd.Series(extract_base_name(x)))

df2_filtered = df2_filtered.sort_values(by='number', ascending=False).drop_duplicates(subset='base_nazev', keep='first').drop(columns=['base_nazev', 'number'])
df_cmp = df2_filtered[pd.to_numeric(df2_filtered['ID'], errors='coerce').notna()]

df_cmp.to_excel('products.xlsx', index=False)


In [2]:
from src.matching import find_products_in_review, generate_phrases
from src.preprocessing import prepare_product_df
import pandas as pd

# Test example
review_text = ("Zastosowaliśmy ARGICIT STAR wraz z Brain STAR po udarze u 60-letniej kobiety, natychmiast po wypisaniu z OIOM-u. Według personelu szpitala rehabilitacja i terapia mowy przebiegają lepiej i skuteczniej niż w przypadku braku leczenia wspomagającego.")

product_df = prepare_product_df(pd.read_excel("products.xlsx"))
find_products_in_review(review_text, product_df)

,ID,nazev,phrase,score
1,1950,BRAIN STAR,brain,1.0
3,7198,ARGICIT STAR,argicit,1.0


In [6]:
for review_text in df['zkusenost'].sample(1000):
    print(f"Review: {review_text}")
    print(find_products_in_review(review_text, product_df))
    print("\n---------------\n")
   

Review: Velmi jsme si zvykla, stejně jako celá moje rodina na používání těchto zubních kartáčků. Krom hojivých a antibakteriálních účinků oceňuji také jejich relativně dlouhou trvanlivost.
Empty DataFrame
Columns: [ID, nazev, phrase, score]
Index: []

---------------

Review: V zimnom období som bola často chorá,chrípky a kašeľ boli na dennom poriadku.Na odporúčanie MUDr.Tomovej som užívala Cordyceps a Chlamynyl Star.Po vybratí dvoch balení sa mi stav zlepšil,odvtedy som v poriadku a som rada,že nemusím užívať atb.
     ID           nazev     phrase  score
1  1888  CORDYCEPS STAR  cordyceps    1.0
2  1102  CHLAMYNYL STAR  chlamynyl    1.0

---------------

Review: Produkt za którego powinni przyznać Nagrodę Nobla. Czas zatrzymał się dla mnie. Widać to po twarzy która wygląda promiennie i młodo.
Empty DataFrame
Columns: [ID, nazev, phrase, score]
Index: []

---------------

Review: Mineral Star svojim bohatym zlozenim sa velmni priaznivo osvedcil_x000D_
v pripade styridsiatnika s diagno

In [3]:
zkusenost, lng, nadpis = df[['zkusenost', 'lng', 'nadpis']].sample(1).iloc[0]
res = await agent({"zkusenost": zkusenost, 'lng': lng, 'nadpis': nadpis})
res['prodID'] = find_products_in_review(res['zkusenost'], product_df)['ID'].astype(str).str.cat(sep=",")
res

{'tagy': 'Blutdruck, Aufrechterhaltung des Blutdrucks, kein Knoblauchgeruch, geschmackvoll',
 'nadpis': 'Knoblauch Petersilie Star Bewertung',
 'zkusenost': 'GARLIC PARSLEY STAR ist wirklich großartig. Es ist eine große Hilfe für mich bei der Aufrechterhaltung meiner Blutdruckwerte und ich schätze es sehr, dass es geschmackvoll ist und nicht typisch nach Knoblauch riecht. Ich empfehle es.',
 'prodID': '7160'}

In [7]:
df_new = df.copy()

for index, row in df_new.iloc[2:5].iterrows():
    print(index)

2
3
4


In [ ]:
import numpy as np
from IPython.display import clear_output
import time
df_new = df.copy()
ipos = 0
for index, row in df_new.iloc[ipos:].iterrows():
    clear_output(wait=True)
    print(f"Progress: {ipos}/{df_new.shape[0]}")
    try:
        res = await agent({"zkusenost": row['zkusenost'], 'lng': row['lng'], 'nadpis': row['nadpis']})
    except Exception as e:
        print(f"{e} - waiting")
        time.sleep(60)
            
    # print(f"Zkusenost:\n{row['zkusenost']}")
    new_prod_ids = find_products_in_review(res['zkusenost'], product_df)['ID'].astype(str).tolist()
    # print(f"New ids: {new_prod_ids}")
    existing_prod_ids = row.get('prodID', "").split(", ") if row.get('prodID') else []
    # print(f"Existing ids: {existing_prod_ids}")
    unique_prod_ids = list(dict.fromkeys(existing_prod_ids + new_prod_ids))
    # print(f"New list of ids: {unique_prod_ids}")

    res['prodID'] = ", ".join(unique_prod_ids)
    df_new.loc[index, ['prodID', 'nadpis', 'tagy']] = [res['prodID'], res['nadpis'], res['tagy']]


0/15
Zkusenost:
Sto per andare in vacanza e la mia attrezzatura non può non includere BETA CAROTEN STAR. Ho iniziato a usare l'1-0-1 due settimane prima delle vacanze, ho sperimentato che non mi scotto al sole, l'abbronzatura è uniforme e dura molto più a lungo.
New ids: ['7110']
Existing ids: ['7110']
New list of ids: ['7110']
1/15
Zkusenost:
MAI-REI SHIITAKE ha ripetutamente dimostrato di avere successo per le clienti che presentano reperti cervicali, citologia elevata o alta. Entro 3 mesi, il 95% delle raccolte di follow-up è migliorato.
New ids: ['5845']
Existing ids: ['5845']
New list of ids: ['5845']
2/15
Zkusenost:
Miałem zapalenie pęcherza. Siostra namówiła mnie na kuracje suplementami. Jestem pod ogromnym wrażeniem szybkości działania tych produktów. Zażywałem: ALFALFA, CRANBERRY, SAW PALMETTO, VILCACORA, ANATOMAX - zapalenia pęcherza moczowego wyleczone. Polecam
New ids: ['2701', '1111', '2739', '2725', '1001']
Existing ids: ['1111']
New list of ids: ['1111', '2701', '2739', 

In [4]:
df_new.iloc[ilocs][['prodID', 'zkusenost']]

,prodID,zkusenost
219,7110,"Sto per andare in vacanza e la mia attrezzatura non può non includere BETA CAROTEN STAR. Ho iniziato a usare l'1-0-1 due settimane prima delle vacanze, ho sperimentato che non mi scotto al sole, l'abbronzatura è uniforme e dura molto più a lungo."
45111,5845,"MAI-REI SHIITAKE ha ripetutamente dimostrato di avere successo per le clienti che presentano reperti cervicali, citologia elevata o alta. Entro 3 mesi, il 95% delle raccolte di follow-up è migliorato."
23153,"1111, 2701, 2739, 2725, 1001","Miałem zapalenie pęcherza. Siostra namówiła mnie na kuracje suplementami. Jestem pod ogromnym wrażeniem szybkości działania tych produktów. Zażywałem: ALFALFA, CRANBERRY, SAW PALMETTO, VILCACORA, ANATOMAX - zapalenia pęcherza moczowego wyleczone. Polecam"
24865,"1841, 1845","Keď sa cítim veľmi unavený, bez energie, a celé moje telo sa cíti zle. Používam Multy star. Po jeho nasadení sa všetky moje problémy odstránia."
39128,"1960, 7292","The product CBD Star Q 10 + E, 33% is excellent for agitated nerves. The combination with Trend relax is very good."
23163,1825,"Acest produs dupa 3 luni de folosire,curata tot sistemul digestiv,regleaza scaunul,dispare constipatia si cantitatea de scaun mai mare"
734,"4521, 2740","B 12 în picături este un supliment absolut excelent pentru persoanele aflate în stres mental, va da calm și va relaxa perfect întregul corp în bunăstare și de la spasmele de stres."
38304,2758,"Покупаю TRAMOSTIC STAR для своего мужа в качестве профилактического средства от простатита и хотя он уже в солидном возрасте, так что в плане урологии все в порядке. Показала врачу, что он принимает, и он похвалил состав, доволен со всех сторон"
25724,7758,Ismételten az eggyik legtöbbet fogyasztott termékröl számolok be. Az ize ősszetétele és hatása levett a lábamrol. Finom lágy és nem savas de selymes ize a benne található ösi hatoanyagok melyek még a fizikális állapotomon is javitanak nem csak nekem hanem a családom többi tagjának is bevált. Ezt a kávét minden honapban fogyasztjuk mert az eddigieknél soskal de sokkal jobb minden tekintetben.
6729,"1570, 2726, 1845","Ca să am un control eficient asupra hipotiroidismului meu ereditar, inițial am folosit un produs care din păcate nu mai există în oferta Starlife: FUCOIDAN STAR. In lipsa acestuia, am folosit KELP STAR, dar din cauza excesului meu de greutate m-am gândit să încerc capsula METABOLITE STAR, care este mai complex, deoarece conține vitamina B6 si oțet de mere pe lângă Kelp. A funcționat așa cum mă așteptam. Mi-a stabilizat metabolismul și m-a ajutat să slăbesc. Funcția tiroidiană a revenit la normal, chiar dacă nu am luat niciodată niciun medicament pentru aceasta. Vă mulțumesc foarte mult pentru această oportunitate ."


In [5]:
df.iloc[ilocs][['prodID', 'zkusenost']]

,prodID,zkusenost
219,7110,"Sto per andare in vacanza e la mia attrezzatura non può non includere BETA CAROTEN STAR. Ho iniziato a usare l'1-0-1 due settimane prima delle vacanze, ho sperimentato che non mi scotto al sole, l'abbronzatura è uniforme e dura molto più a lungo."
45111,5845,"MAI-REI SHIITAKE ha ripetutamente dimostrato di avere successo per le clienti che presentano reperti cervicali, citologia elevata o alta. Entro 3 mesi, il 95% delle raccolte di follow-up è migliorato."
23153,1111,"Miałem zapalenie pęcherza. Siostra namówiła mnie na kuracje suplementami. Jestem pod ogromnym wrażeniem szybkości działania tych produktów. Zażywałem: ALFALFA, CRANBERRY, SAW PALMETTO, VILCACORA, ANATOMAX - zapalenia pęcherza moczowego wyleczone. Polecam"
24865,1841,"Keď sa cítim veľmi unavený, bez energie, a celé moje telo sa cíti zle. Používam Multy star. Po jeho nasadení sa všetky moje problémy odstránia."
39128,1960,"The product CBD Star Q 10 + E, 33% is excellent for agitated nerves. The combination with Trend relax is very good."
23163,1825,"Acest produs dupa 3 luni de folosire,curata tot sistemul digestiv,regleaza scaunul,dispare constipatia si cantitatea de scaun mai mare"
734,4521,"B 12 în picături este un supliment absolut excelent pentru persoanele aflate în stres mental, va da calm și va relaxa perfect întregul corp în bunăstare și de la spasmele de stres."
38304,2758,"Покупаю TRAMOSTIC STAR для своего мужа в качестве профилактического средства от простатита и хотя он уже в солидном возрасте, так что в плане урологии все в порядке. Показала врачу, что он принимает, и он похвалил состав, доволен со всех сторон"
25724,7758,Ismételten az eggyik legtöbbet fogyasztott termékröl számolok be. Az ize ősszetétele és hatása levett a lábamrol. Finom lágy és nem savas de selymes ize a benne található ösi hatoanyagok melyek még a fizikális állapotomon is javitanak nem csak nekem hanem a családom többi tagjának is bevált. Ezt a kávét minden honapban fogyasztjuk mert az eddigieknél soskal de sokkal jobb minden tekintetben.
6729,1570,"Ca să am un control eficient asupra hipotiroidismului meu ereditar, inițial am folosit un produs care din păcate nu mai există în oferta Starlife: FUCOIDAN STAR. In lipsa acestuia, am folosit KELP STAR, dar din cauza excesului meu de greutate m-am gândit să încerc capsula METABOLITE STAR, care este mai complex, deoarece conține vitamina B6 si oțet de mere pe lângă Kelp. A funcționat așa cum mă așteptam. Mi-a stabilizat metabolismul și m-a ajutat să slăbesc. Funcția tiroidiană a revenit la normal, chiar dacă nu am luat niciodată niciun medicament pentru aceasta. Vă mulțumesc foarte mult pentru această oportunitate ."


In [10]:
from src.validate import check_df_integrity
check_df_integrity(df, df_new)

{'status': 'Integrity check passed'}

In [ ]:
show_ids = ['23153']